# Artificial Neural Network

## Definition:
* Computational graph model based on the structure and function of biological neural networks. 
* Consist of layers of interconnected processors (nodes) called **neurons**.
* Information is stored in the **synapses** (weights) of the connection carried through the **axons** (links).
* Layers of neurons learns by performing *summing functions* of the weighted-sums of the rows of input 'signals' and produces a predictory output signal, much like how a neuron *activates* after sensing various stimuli.
* Learning is done by *backpropagating* through the network where the weights are then adjusted to minimise the error in prediction, determined by some *loss function*, using an optimisation strategy which is typically *Gradient descent*.

## The Layers
* Input layer: where all the inputs are fed into the model.
* Hidden layers: where the processing/computation happens.
* Output layer: where the final result(s) of the computation are outputted.

<img src="../_images/ANN.png" alt="ANN" width="500"/>
    

## Activation Function 
Function that determines activation of a neuron (output-value per neuron).

$$
f\left(\sum_{i=i}^{N} w_{i}x_{i}\right)
$$

where $\sum_{i=i}^{N} w_{i}x_{i}$ is the weighted sum of all the weights from the synapes connected to that perticular neuron, for $N$ rows of input signals $x_{i}$.

* ### Info
    * Allows non-linear mapping between input-output pairs. If we don't apply an activation function then the mapping will always be linear.
    * Should be differentiable, so we can backpropagate in the network to compute gradient of errors.

* ### Types
    * **Sigmoid**: $$f(x) = \frac{1}{1+e^{-x}}$$.
        * Also known as the **Logistic** function. 
        * Ranges from [0,1].
        * 'S'-shaped curve function of the form
        * Not zeroed centred (makes the gradient updates go too far in different direction, making optimisation harder.
        * Saturate and kill gradients.
        * Slow convergence (have exponential operation)
    * **Tanh:** $$f(x) = \frac{1-e^{-2x}}{1+e^{2x}}$$
        * Ranges from [-1,1]
        * A zero-centered version of Sigmoid/Logistic 
    * **ReLu:** $$f(x) = \max(x,0)$$
        * if $x<0$ then $f(x)=0$
        * if $x\geq0$ then $f(x)=x$
        * does not suffer from Vanishing gradient problem like Sigmoid/Tanh
        * should only be used within hidden layers
        * Faster in convergence
        
<img src="../_images/activations.png" alt="act" width="800"/>
    
Read more: [activation-functions-towards-data-science](https://towardsdatascience.com/activation-functions-and-its-types-which-is-better-a9a5310cc8f) 
[introduction-activation](https://medium.com/@shrutijadon10104776/survey-on-activation-functions-for-deep-learning-9689331ba092)

## Loss Function
* ### Mean Squared
$$
L = \frac{1}{N}\sum_{i=1}^{N}(y - \hat{y})^{2}
$$

* ### L2
$$
L = \sum_{i=1}^{N}(y - \hat{y})^{2}
$$
read more: [isaac_changhau](https://isaacchanghau.github.io/post/loss_functions/)
[loss-functions-dymestified](https://medium.com/data-science-group-iitr/loss-functions-and-optimization-algorithms-demystified-bb92daff331c)
[arxiv.org](https://arxiv.org/pdf/1702.05659.pdf)

## Gradient Descent
Optimisation technique reduce errors. The loss function should be convex, if not convex, use Stochastic gradient descent to avoid 'zig-zaggy'convergence to a local minima, i.e. to better ensure global minima can be reach instead.

* ### Batch Gradient Descent
update weights after accounting all rows of observations.


* ### Stochastic Gradient Descent
update weights per iteration.
    * 1.) randomly initialise the weights to small numbers close to 0
    * 2.) input first observation of dataset in the input layer, each feature in one input node
    * 3.) forward progate the neurons to get predicted result y
    * 4.) compare predicted result to the actual result. measure generated error
    * 5.) backpropagate, update the weights according to how much they are responsible for the error. The learning rate decides by how much we update them.
    * 6.) Repeat steps 1.) to 5.)
    * 7.) When the whole training set passed through the ANN, that makes an epoch, redo more epochs.
    
read more: [gradient_descent_nutsell](https://towardsdatascience.com/gradient-descent-in-a-nutshell-eaf8c18212f0)
[backpropagation](https://towardsdatascience.com/back-propagation-414ec0043d7)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
import warnings
'''
These warnings filter are to unclutter output and easy-to-read purposes only!
Remember to lookup replacements. Uncomment for info
Reminder: some functions in sckitlearn have been/will be depreceated,
'''
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

## Data Pre-Processing

In [9]:
warnings.simplefilter(action='ignore', category=DeprecationWarning)
dataset = pd.read_csv('../_data/Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# dealing with categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# splitting into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 0)

# feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

print('Dimensions of dataset:', dataset.shape)
dataset.head() # put dataset.head(datase.shape[0]) to see all

Dimensions of dataset: (10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Training & Testing

In [10]:
# initialising the ANN
classifier = Sequential()

# adding the input layer and the first hidden layer
classifier.add(Dense(
        units = 6,
        kernel_initializer = 'uniform',
        activation = 'relu',
        input_dim = 11))
# adding the second hidden layer
classifier.add(Dense(
        units = 6,
        kernel_initializer = 'uniform',
        activation = 'relu'))
# adding the output layer
classifier.add(Dense(
        units = 1,
        kernel_initializer = 'uniform',
        activation = 'sigmoid'))
# compiling the ANN
classifier.compile(optimizer = 'adam',
                   loss = 'binary_crossentropy',
                   metrics = ['accuracy'])

# fitting to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

# predicting using the Test set
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
CM = pd.DataFrame(cm)
CM.head()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4773 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 121us/step - loss: 0.4306 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4265 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 98us/step - loss: 0.4211 - acc: 0.8072
Epoch 5/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4179 - acc: 0.8235
Epoch 6/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4158 - acc: 0.8279
Epoch 7/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4140 - acc: 0.8306
Epoch 8/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4125 - acc: 0.8309
Epoch 9/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.

8000/8000 [==============================] - 1s 113us/step - loss: 0.4000 - acc: 0.8364
Epoch 77/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4000 - acc: 0.8365
Epoch 78/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4001 - acc: 0.8346
Epoch 79/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4002 - acc: 0.8355
Epoch 80/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4003 - acc: 0.8360
Epoch 81/100
8000/8000 [==============================] - 1s 94us/step - loss: 0.4004 - acc: 0.8367
Epoch 82/100
8000/8000 [==============================] - 1s 143us/step - loss: 0.4004 - acc: 0.8357 0s - loss: 0.4020 - acc: 0.
Epoch 83/100
8000/8000 [==============================] - 1s 128us/step - loss: 0.4005 - acc: 0.8369
Epoch 84/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.4000 - acc: 0.8346
Epoch 85/100
8000/8000 [==============================] - 1s 91us/step - loss:

,0,1
0,1547,48
1,271,134


## Evaluation & Tuning

In [11]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(
            units = 6, 
            kernel_initializer = 'uniform', 
            activation = 'relu', 
            input_dim = 11))
    classifier.add(Dense(
            units = 6,
            kernel_initializer = 'uniform',
            activation = 'relu'))
    classifier.add(Dense(
            units = 1, 
            kernel_initializer = 'uniform', 
            activation = 'sigmoid'))
    classifier.compile(
            optimizer = 'adam', 
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier,
                             batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, 
                             cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Improving and Tuning the ANN
# Dropout Regularization to reduce overfitting if needed

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, 
                         kernel_initializer = 'uniform',
                         activation = 'relu', 
                         input_dim = 11))
    classifier.add(Dense(units = 6, 
                         kernel_initializer = 'uniform', 
                         activation = 'relu'))
    classifier.add(Dense(units = 1, 
                         kernel_initializer = 'uniform', 
                         activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, 
                       loss = 'binary_crossentropy', 
                       metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

KeyboardInterrupt: 